# Exercises 4.

*Antti Härkönen*

## 1.


In [19]:
import numpy as np
import torch
import torchvision.datasets as dset
import torch.nn as nn
import torchvision.transforms as transforms
import pyro
import pyro.distributions as dist
import pyro.contrib.examples.util
from pyro.infer import Trace_ELBO, SVI
from pyro.optim import Adam
import matplotlib.pyplot as plt
import visdom
from sklearn.manifold import TSNE

root = './data'
transform = transforms.ToTensor()
train_data = dset.MNIST(root=root, train=True, transform=transform, download=True)
test_data = dset.MNIST(root=root, train=False, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=100, shuffle=True)


class Decoder(nn.Module):
    def __init__(self, z_dim, hidden_dim):
        super().__init__()
        self.fc1 = nn.Linear(z_dim, hidden_dim)
        self.fc21 = nn.Linear(hidden_dim, 784)
        self.softplus = nn.Softplus()
        self.sigmoid = nn.Sigmoid()

    def forward(self, z):
        hidden = self.softplus(self.fc1(z))
        loc_img = self.sigmoid(self.fc21(hidden))
        return loc_img


class Encoder(nn.Module):
    def __init__(self, z_dim, hidden_dim):
        super().__init__()
        self.fc1 = nn.Linear(784, hidden_dim)
        self.fc21 = nn.Linear(hidden_dim, z_dim)
        self.fc22 = nn.Linear(hidden_dim, z_dim)
        self.softplus = nn.Softplus()

    def forward(self, x):
        x = x.reshape(-1, 784)
        hidden = self.softplus(self.fc1(x))
        z_loc = self.fc21(hidden)
        z_scale = torch.exp(self.fc22(hidden))
        return z_loc, z_scale


class VAE(nn.Module):
    def __init__(
            self,
            z_dim=50,
            hidden_dim=400,
            encoder=Encoder,
            decoder=Decoder,
            use_cuda=False,
    ):
        super().__init__()
        self.encoder = encoder(z_dim, hidden_dim)
        self.decoder = decoder(z_dim, hidden_dim)

        if use_cuda:
            self.cuda()
        self.use_cuda = use_cuda
        self.z_dim = z_dim

    def model(self, x):
        pyro.module("decoder", self.decoder)
        with pyro.plate("data", x.shape[0]):
            z_loc = torch.zeros(x.shape[0], self.z_dim, dtype=x.dtype, device=x.device)
            z_scale = torch.ones(x.shape[0], self.z_dim, dtype=x.dtype, device=x.device)
            z = pyro.sample("latent", dist.Normal(z_loc, z_scale).to_event(1))
            loc_img = self.decoder.forward(z)
            pyro.sample(
                "obs",
                dist.Bernoulli(loc_img, validate_args=False).to_event(1),
                obs=x.reshape(-1, 784),
            )
            return loc_img

    def guide(self, x):
        pyro.module("encoder", self.encoder)
        with pyro.plate("data", x.shape[0]):
            z_loc, z_scale = self.encoder.forward(x)
            pyro.sample("latent", dist.Normal(z_loc, z_scale).to_event(1))

    def predict_img(self, x):
        z_loc, z_scale = self.encoder(x)
        z = dist.Normal(z_loc, z_scale).sample()
        loc_img = self.decoder(z)
        return loc_img


def train(svi, train_loader, use_cuda=False):
    epoch_loss = 0.
    for x, _ in train_loader:
        if use_cuda:
            x = x.cuda()
        epoch_loss += svi.step(x)
    normalizer = len(train_loader.dataset)
    loss = epoch_loss / normalizer
    return loss


def plot_vae_samples(vae, visdom_session):
    vis = visdom_session
    x = torch.zeros([1, 784])
    for i in range(10):
        images = []
        for rr in range(100):
            sample_loc_i = vae.model(x)
            img = sample_loc_i[0].view(1, 28, 28).cpu().data.numpy()
            images.append(img)
        vis.images(images, 10, 2)


def mnist_test_tsne(vae=None, test_loader=None):
    data = test_loader.dataset.test_data.float()
    mnist_labels = test_loader.dataset.test_labels
    z_loc, z_scale = vae.encoder(data)
    plot_tsne(z_loc, mnist_labels, 'vae')


def plot_tsne(z_loc, classes, name):
    model_tsne = TSNE(n_components=2, random_state=0)
    z_states = z_loc.detach().cpu().numpy()
    z_embed = model_tsne.fit_transform(z_states)
    classes = classes.detach().cpu().numpy()
    fig = plt.figure()
    for ic in range(10):
        ind_vec = np.zeros_like(classes)
        ind_vec[:, ic] = 1
        ind_class = classes[:, ic] == 1
        color = plt.cm.Set1(ic)
        plt.scatter(z_embed[ind_class, 0], z_embed[ind_class, 1], s=10, color=color)
        plt.title("Latent Variable T-SNE per Class")
        fig.savefig('./vae_results/'+str(name)+'_embedding_'+str(ic)+'.png')
    fig.savefig('./vae_results/'+str(name)+'_embedding.png')

use_cuda = False
vae = VAE(use_cuda=use_cuda)
optimizer = Adam({"lr": 0.01})
vis = visdom.Visdom()

svi = SVI(vae.model, vae.guide, optimizer, loss=Trace_ELBO())

train_elbo = []
for epoch in range(10):
    loss = train(svi, train_loader, use_cuda=use_cuda)
    train_elbo.append(-loss)
    print(f"Epoch {epoch} training loss: {loss:.3f}")

plot_vae_samples(vae, vis)
for x, _ in test_loader:
    reco_indices = np.random.randint(0, x.shape[0], 3)
    for index in reco_indices:
        test_img = x[index, :]
        reco_img = vae.predict_img(test_img)
        vis.image(test_img.reshape(28, 28).detach().cpu().numpy(),
                  opts={'caption': 'test image'})
        vis.image(reco_img.reshape(28, 28).detach().cpu().numpy(),
                  opts={'caption': 'reconstructed image'})

Setting up a new session...
Traceback (most recent call last):
  File "C:\Users\antth\AppData\Roaming\Python\Python38\site-packages\urllib3\connection.py", line 169, in _new_conn
    conn = connection.create_connection(
  File "C:\Users\antth\AppData\Roaming\Python\Python38\site-packages\urllib3\util\connection.py", line 96, in create_connection
    raise err
  File "C:\Users\antth\AppData\Roaming\Python\Python38\site-packages\urllib3\util\connection.py", line 86, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] Yhteyttä ei voi muodostaa, koska kohdekone ei salli sitä

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\antth\AppData\Roaming\Python\Python38\site-packages\urllib3\connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "C:\Users\antth\AppData\Roaming\Python\Python38\site-packages\urllib3\connectionpool.py", line 394, in _make_request
 

Exception in user code:
------------------------------------------------------------
Epoch 0 training loss: 590.934
Epoch 1 training loss: 590.896
Epoch 2 training loss: 590.934
Epoch 3 training loss: 590.924
Epoch 4 training loss: 590.905
Epoch 5 training loss: 590.944
Epoch 6 training loss: 590.929
Epoch 7 training loss: 590.910
Epoch 8 training loss: 590.865
Epoch 9 training loss: 590.927
Exception in user code:
------------------------------------------------------------
Exception in user code:
------------------------------------------------------------
Exception in user code:
------------------------------------------------------------
Exception in user code:
------------------------------------------------------------
Exception in user code:
------------------------------------------------------------
Exception in user code:
------------------------------------------------------------
Exception in user code:
------------------------------------------------------------
Exception 

TypeError: object of type 'NoneType' has no len()

## 2.


## 3.


## 4.

### Shengjia Zhao, Jiaming Song, Stefano Ermon 2018: InfoVAE: Balancing Learning and Inference in Variational Autoencoders

The goal of the work is to find a new variational autoencoder inference method to avoid the problems of ELBO.
Inference measure is calculated using reparameterization trick, but there is a term that must be estimated by sampling and optimizing.
The method worked better than ELBO in a comparison using the MNIST digit dataset.
